# Graded Assignment 3: 9 to 5

Time to show off your SQL skills! For each question, copy the SQL query you used and make note of the answer.

## The Dataset

For this assignment, you will be using the Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results which can be found on [Kaggle](https://www.kaggle.com/datasets/bls/employment).

## Business Issue

You are working for the Bureau of Labor Statistics with the United States government and have been approached by your boss with an important meeting request. You have been asked by your supervisor to meet with Dolly Parton whose nonprofit is looking to shed light on the state of employment in the United States. As part of the 9 to 5 project, their research is focused on production and nonsupervisory employees and how those employees fare compared to all employees in the United States. While the data the BLS collects from the CES is publicly available, Dolly Parton and her colleagues need your assistance navigating the thousands of rows in each table in LaborStatisticsDB.

## About the Dataset

This dataset comes directly from the Bureau of Labor Statistics’ Current Employment Survey (CES). Here are some things you need to know:

1. The industry table contains an NAICS code. This is different from the industry code. NAICS stands for North American Industry Classification System.
1. Series ID is composed of multiple different codes. CES stands for Current Employment Survey, the name of the survey which collected the data. The industry code as specified by the BLS and the data type code as specified in the datatype table.

## Set Up

To connect to the database, use the same connection info used during the SQL lessons. 

For the assignment, we will be using `LaborStatisticsDB`.

## Database Exploration

To start with, let’s get to know the database further.

1. Use this space to make note of each table in the database, the columns within each table, each column’s data type, and how the tables are connected. You can write this down or draw a diagram. Whatever method helps you get an understanding of what is going on with `LaborStatisticsDB`.
   
   To add a photo, diagram or document to your file, drop the file into the folder that holds this notebook.  Use the link button to the right of the  </> symbol in the gray part of this cell, the link is just the name of your file.

In [ ]:
SELECT *
FROM LaborStatisticsDB.dbo.annual_2016
/* 
annual_2016

ID INT
year INT
period VARCHAR
value INT
footnote_codes VARCHAR
original_file VARCHAR
*/

SELECT *
FROM LaborStatisticsDB.dbo.datatype
/*
datatype

data_type_code INT
data_type_text VARCHAR
*/

SELECT *
FROM LaborStatisticsDB.dbo.footnote
/*
footnote

footnote_code VARCHAR
footnote_text VARCHAR
*/

SELECT *
FROM LaborStatisticsDB.dbo.industry
/*
industry 

id INT
industry_code INT
naics_code VARCHAR
publishing_status VARCHAR
industry_name VARCHAR
display_level INT
selectable VARCHAR 
sort_sequenceINT
*/

SELECT *
FROM LaborStatisticsDB.dbo.january_2017
/*
january_2017

id INT
series_id VARCHAR
year INT
period VARCHAR
value INT
footnote_codes VARCHAR
original_file VARCHAR
*/

SELECT *
FROM LaborStatisticsDB.dbo.[period]
/*
period

period_code VARCHAR
month_abbr VARCHAR
month VARCHAR
*/

SELECT *
FROM LaborStatisticsDB.dbo.seasonal
/*
seasonal 

industry_code VARCHAR
seasonal_text VARCHAR
*/

SELECT *
FROM LaborStatisticsDB.dbo.series
/* 
series

series_id VARCHAR 
supersector_code INT
industry_code INT
data_type_code INT
seasonal VARCHAR
series_title VARCHAR
*/

SELECT *
FROM LaborStatisticsDB.dbo.supersector
/*
supersector

supersector_code INT
supersector_name VARCHAR
*/ 

2. What is the datatype for women employees?

In [ ]:
SELECT dt.data_type_code, dt.data_type_text
FROM LaborStatisticsDB.dbo.datatype AS dt
WHERE dt.data_type_text LIKE '%WOMEN%'

/*
Code: 10	Text: WOMEN EMPLOYEES
Code: 39	Text: WOMEN EMPLOYEES-TO-ALL EMPLOYEES RATIO
*/

3. What is the series id for  women employees in the commercial banking industry in the financial activities supersector?

In [ ]:
SELECT s.series_id
FROM LaborStatisticsDB.dbo.series AS s
JOIN LaborStatisticsDB.dbo.datatype AS dt ON s.data_type_code = dt.data_type_code
JOIN LaborStatisticsDB.dbo.industry AS i ON s.industry_code = i.industry_code
JOIN LaborStatisticsDB.dbo.supersector AS ss ON s.supersector_code = ss.supersector_code
WHERE 
    ss.supersector_name LIKE '%Financial Activities%'
    AND i.industry_name LIKE '%Commercial banking%'
    AND dt.data_type_text = 'WOMEN EMPLOYEES'

-- The series ID is CES5552211010

## Aggregate Your Friends and Code some SQL

Put together the following:

1. How many employees were reported in 2016 in all industries? Round to the nearest whole number.

In [ ]:
SELECT ROUND(SUM(ann.value),0)
FROM LaborStatisticsDB.dbo.annual_2016 AS ann
JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = ann.series_id
JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code = s.data_type_code
WHERE dt.data_type_text LIKE 'ALL EMPLOYEES'
-- ANSWER: 2340612

2. How many women employees were reported in 2016 in all industries? Round to the nearest whole number. 

In [ ]:

SELECT ROUND(SUM(ann.[value]),0)
FROM LaborStatisticsDB.dbo.annual_2016 AS ann
JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = ann.series_id
JOIN LaborStatisticsDB.dbo.industry AS i ON i.industry_code = s.industry_code
JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code = s.data_type_code
WHERE dt.data_type_text LIKE 'WOMEN EMPLOYEES'
    AND i.industry_name IS NOT NULL
-- ANSWER: 1093094

3. How many production/nonsupervisory employees were reported in 2016? Round to the nearest whole number. 

In [ ]:

SELECT ROUND(SUM(ann.[value]),0)
FROM LaborStatisticsDB.dbo.annual_2016 AS ann
JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = ann.series_id
JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code = s.data_type_code
WHERE dt.data_type_text LIKE 'PRODUCTION AND NONSUPERVISORY EMPLOYEES'
-- 1263650

4. In January 2017, what is the average weekly hours worked by production and nonsupervisory employees across all industries?

In [ ]:
SELECT ROUND(AVG(jan.VALUE),0)
FROM LaborStatisticsDB.dbo.january_2017 AS jan
JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = jan.series_id
WHERE s.series_title LIKE '%Average weekly hours of production and nonsupervisory employees%'

-- Average weekly hours: 36

5. What is the total weekly payroll for production and nonsupervisory employees across all industries in January 2017? Round to the nearest penny.

In [ ]:
SELECT SUM(jan.[value])
FROM LaborStatisticsDB.dbo.january_2017 AS jan 
JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = jan.series_id
JOIN LaborStatisticsDB.dbo.industry AS i ON i.industry_code = s.industry_code
JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code = s.data_type_code
WHERE dt.data_type_code = 82
    AND i.industry_name IS NOT NULL
-- less confident in this one! but I got the answer: 1769253164

6. In January 2017, for which industry was the average weekly hours worked by production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
SELECT jan.[value], dt.data_type_text, i.industry_name
FROM LaborStatisticsDB.dbo.industry AS i 
JOIN LaborStatisticsDB.dbo.series AS s ON s.industry_code = i.industry_code
JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code = s.data_type_code
JOIN LaborStatisticsDB.dbo.january_2017 AS jan ON jan.series_id = s.series_id
WHERE dt.data_type_text LIKE 'AVERAGE WEEKLY HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES'
ORDER BY jan.[value] DESC
-- ANSWER: The highest - Motor vehicle power train components

SELECT jan.[value], dt.data_type_text, i.industry_name
FROM LaborStatisticsDB.dbo.industry AS i 
JOIN LaborStatisticsDB.dbo.series AS s ON s.industry_code = i.industry_code
JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code = s.data_type_code
JOIN LaborStatisticsDB.dbo.january_2017 AS jan ON jan.series_id = s.series_id
WHERE dt.data_type_text LIKE 'AVERAGE WEEKLY HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES'
ORDER BY jan.[value] ASC
-- ANSWER: The lowest - Fitness and recreational sports centers

7. In January 2021, for which industry was the total weekly payroll for production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
SELECT jan.[value], dt.data_type_text, i.industry_name
FROM LaborStatisticsDB.dbo.industry AS i 
JOIN LaborStatisticsDB.dbo.series AS s ON s.industry_code = i.industry_code
JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code = s.data_type_code
JOIN LaborStatisticsDB.dbo.january_2017 AS jan ON jan.series_id = s.series_id
WHERE dt.data_type_text LIKE 'AGGREGATE WEEKLY PAYROLLS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES'
    AND i.industry_name IS NOT NULL
ORDER BY jan.[value] DESC
-- ANSWER: The highest - Total Private (Private service-providing)

SELECT jan.[value], dt.data_type_text, i.industry_name
FROM LaborStatisticsDB.dbo.industry AS i 
JOIN LaborStatisticsDB.dbo.series AS s ON s.industry_code = i.industry_code
JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code = s.data_type_code
JOIN LaborStatisticsDB.dbo.january_2017 AS jan ON jan.series_id = s.series_id
WHERE dt.data_type_text LIKE 'AGGREGATE WEEKLY PAYROLLS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES'
    AND i.industry_name IS NOT NULL
ORDER BY jan.[value] ASC
-- Answer: The lowest - Coin-operated laundries and drycleaners

## Join in on the Fun

Time to start joining! You can choose the type of join you use, just make sure to make a  note!

1. Join `annual_2016` with `series` on `series_id`. We only want the data in the `annual_2016` table to be included in the result.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
SELECT TOP 50 *
FROM LaborStatisticsDB.dbo.annual_2016 AS ann 
JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = ann.series_id


-- Uncomment the line below when you are ready to run the query, leaving it as your last!
SELECT TOP 50 *
FROM LaborStatisticsDB.dbo.annual_2016 AS ann 
JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = ann.series_id
ORDER BY id

Commands completed successfully.

Total execution time: 00:00:00.019

2. Join `series` and `datatype` on `data_type_code`.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
SELECT TOP 50 *
FROM LaborStatisticsDB.dbo.series AS s  
JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code = s.data_type_code

-- Uncomment the line below when you are ready to run the query, leaving it as your last!
SELECT TOP 50 *
FROM LaborStatisticsDB.dbo.series AS s  
JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code = s.data_type_code
ORDER BY s.series_id

3. Join `series` and `industry` on `industry_code`.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
SELECT TOP 50 *
FROM LaborStatisticsDB.dbo.series AS s  
JOIN LaborStatisticsDB.dbo.industry AS i ON i.industry_code = s.industry_code


-- Uncomment the line below when you are ready to run the query, leaving it as your last!
SELECT TOP 50 *
FROM LaborStatisticsDB.dbo.series AS s  
JOIN LaborStatisticsDB.dbo.industry AS i ON i.industry_code = s.industry_code
ORDER BY i.id

## Subqueries, Unions, Derived Tables, Oh My!

1. Write a query that returns the `series_id`, `industry_code`, `industry_name`, and `value` from the `january_2017` table but only if that value is greater than the average value for `annual_2016` of `data_type_code` 82.

In [ ]:
WITH one AS (
    SELECT jan.series_id, i.industry_code, i.industry_name, jan.[value]
    FROM LaborStatisticsDB.dbo.january_2017 AS jan
    JOIN LaborStatisticsDB.dbo.industry AS i ON i.id = jan.id
), 
two AS (
    SELECT ROUND(AVG(ann.[value]), 0) AS annual_average
    FROM LaborStatisticsDB.dbo.annual_2016 AS ann
    JOIN LaborStatisticsDB.dbo.series AS s2 ON s2.series_id = ann.series_id
    JOIN LaborStatisticsDB.dbo.datatype AS dt2 ON dt2.data_type_code = s2.data_type_code
    WHERE dt2.data_type_code = 82
)
SELECT one.series_id, one.industry_code, one.industry_name, one.[value]
FROM one
WHERE one.[value] > (SELECT annual_average FROM two)

**Optional Bonus Question:** Write the above query as a common table expression!

In [ ]:
-- Optional CTE below
-- WHoops! Already did that! 
WITH one AS (
    SELECT jan.series_id, i.industry_code, i.industry_name, jan.[value]
    FROM LaborStatisticsDB.dbo.january_2017 AS jan
    JOIN LaborStatisticsDB.dbo.industry AS i ON i.id = jan.id
), 
two AS (
    SELECT ROUND(AVG(ann.[value]), 0) AS annual_average
    FROM LaborStatisticsDB.dbo.annual_2016 AS ann
    JOIN LaborStatisticsDB.dbo.series AS s2 ON s2.series_id = ann.series_id
    JOIN LaborStatisticsDB.dbo.datatype AS dt2 ON dt2.data_type_code = s2.data_type_code
    WHERE dt2.data_type_code = 82
)
SELECT one.series_id, one.industry_code, one.industry_name, one.[value]
FROM one
WHERE one.[value] > (SELECT annual_average FROM two)

2. Create a `Union` table comparing average weekly earnings of production and nonsupervisory employees between `annual_2016` and `january_2017` using the data type 30.  Round to the nearest penny.  You should have a column for the average earnings and a column for the year, and the period.

In [ ]:
SELECT ROUND(ann.[value],2), ann.[year], ann.[period] FROM LaborStatisticsDB.dbo.annual_2016 AS ann
UNION
SELECT jan.[value], jan.[year], jan.[period] FROM LaborStatisticsDB.dbo.january_2017 AS jan 
JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = jan.series_id
JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code = s.data_type_code
WHERE dt.data_type_code = 30


## Summarize Your Results

With what you know now about the  Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results and working with the Labor Statistics Database, answer the following questions. Note that while this is subjective, you should include relevant data to back up your opinion.

1. During which time period did production and nonsupervisory employees fare better?

Production and nonsupervisory employees fared better in 2017. From 2016 to 2017, on average hourly earnings, weekly earnings, and weekly hours stayed the same. However, for production and non-supervisory employees, hourly earnings increased by $2 (approximately 13%). 

All amployees for 2016:

SELECT ROUND(AVG(ann.[value]), 0), s.series_title
FROM LaborStatisticsDB.dbo.annual_2016 AS ann 
JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = ann.series_id
JOIN LaborStatisticsDB.dbo.[period] AS p ON p.period_code = ann.[period] 
WHERE s.series_title IN ( 
    SELECT DISTINCT s.series_title
    FROM LaborStatisticsDB.dbo.series AS s
    INTERSECT
    SELECT DISTINCT dt.data_type_text
    FROM LaborStatisticsDB.dbo.datatype AS dt
    WHERE s.series_title LIKE '%ALL EMPLOYEES%')
GROUP BY s.series_title
ORDER BY s.series_title ASC
-- Average of average hourly earnings 17
-- Average of average weekly earnings 619
-- Average of average weekly hours 36

Productiona and nonsupervisory employees for 2016:

SELECT ROUND(AVG(ann.[value]),0), s.series_title
FROM LaborStatisticsDB.dbo.annual_2016 AS ann 
JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = ann.series_id
JOIN LaborStatisticsDB.dbo.[period] AS p ON p.period_code = ann.[period] 
WHERE s.series_title IN ( 
    SELECT DISTINCT s.series_title
    FROM LaborStatisticsDB.dbo.series AS s
    INTERSECT
    SELECT DISTINCT dt.data_type_text
    FROM LaborStatisticsDB.dbo.datatype AS dt
    WHERE s.series_title LIKE '%Production and nonsupervisory employees%')
GROUP BY s.series_title 

-- Average hourly earnings 15
-- Average weekly earnings 523
-- Average weekly hours 36

All employees for 2017:

SELECT ROUND(AVG(jan.[value]), 0), s.series_title
FROM LaborStatisticsDB.dbo.january_2017 AS jan   
JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = jan.series_id
WHERE s.series_title IN (
        SELECT DISTINCT s.series_title
        FROM LaborStatisticsDB.dbo.series AS s    
        INTERSECT 
        SELECT DISTINCT dt.data_type_text
        FROM LaborStatisticsDB.dbo.datatype AS dt
        WHERE s.series_title LIKE '%ALL EMPLOYEES%'
)
GROUP BY s.series_title
ORDER BY s.series_title ASC

-- Average hourly earnings 17
-- Average weekly earnings 629
-- Average weekly hours 36


Production and nonsupervisory roles for 2017:
SELECT ROUND(AVG(jan.VALUE), 0), s.series_title, 
FROM LaborStatisticsDB.dbo.january_2017 AS jan   
JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = jan.series_id
WHERE s.series_title IN (
        SELECT DISTINCT s.series_title
        FROM LaborStatisticsDB.dbo.series AS s    
        INTERSECT 
        SELECT DISTINCT dt.data_type_text
        FROM LaborStatisticsDB.dbo.datatype AS dt
        WHERE s.series_title LIKE '%PRODUCTION AND NONSUPERVISORY EMPLOYEES%'
)
GROUP BY s.series_title
ORDER BY s.series_title ASC

/*
-- Average hourly earnings 15
-- Average weekly earnings 528
-- Average weekly hours 36

2016 all employees : 2017 all employees
average of average hourly earnings | 17 : 17
average of average weekly earnings | 619 : 629
average of average weekly hours | 36 : 36


2016 production and nonsupervisory employees : 2017 production and nonsupervisory employees
average of average hourly earnings | 15 : 17
average of average weekly earnings | 523 : 528
average of average weekly hours | 36 : 36


2016 all employees : 2016 production and nonsupervisory employees 
average of average hourly earnings | 17 : 15
average of average weekly earnings | 619 : 523
average of average weekly hours | 36 : 36


2017 all employees : 2017 production and nonsupervisory employees 
average of average hourly earnings | 17 : 15
average of average weekly earnings | 629 : 528
average of average weekly hours | 36 : 36
*/


2. In which industries did production and nonsupervisory employees fare better?

WITH pan AS(
    SELECT 
        i.industry_name,
        AVG(an.[value]) AS pan_avg
    FROM LaborStatisticsDB.dbo.annual_2016 AS an   
    JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = an.series_id
    JOIN LaborStatisticsDB.dbo.industry AS i ON i.industry_code = s.industry_code
    WHERE s.series_title LIKE '%PRODUCTION AND NONSUPERVISORY%'
        AND i.industry_name IS NOT NULL
    GROUP BY i.industry_name
),
ae AS(
    SELECT 
        i.industry_name,
        AVG(an.[value]) AS ae_avg
    FROM LaborStatisticsDB.dbo.annual_2016 AS an   
    JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = an.series_id
    JOIN LaborStatisticsDB.dbo.industry AS i ON i.industry_code = s.industry_code
    WHERE s.series_title LIKE '%ALL EMPLOYEES%'
        AND i.industry_name IS NOT NULL
    GROUP BY i.industry_name
)
    SELECT pan.industry_name
    FROM pan
    JOIN ae ON ae.industry_name = pan.industry_name
    WHERE pan_avg > ae_avg
Production and nonsupervisory employees did better on average in motor vehicle power train components than all employees in 2016 

WITH pan AS(
    SELECT 
        i.industry_name,
        AVG(jan.[value]) AS pan_avg
    FROM LaborStatisticsDB.dbo.january_2017 AS jan  
    JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = jan.series_id
    JOIN LaborStatisticsDB.dbo.industry AS i ON i.industry_code = s.industry_code
    WHERE s.series_title LIKE '%PRODUCTION AND NONSUPERVISORY%'
        AND i.industry_name IS NOT NULL
    GROUP BY i.industry_name
),
ae AS(
    SELECT 
        i.industry_name,
        AVG(jan.[value]) AS ae_avg
    FROM LaborStatisticsDB.dbo.january_2017 AS jan  
    JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = jan.series_id
    JOIN LaborStatisticsDB.dbo.industry AS i ON i.industry_code = s.industry_code
    WHERE s.series_title LIKE '%ALL EMPLOYEES%'
        AND i.industry_name IS NOT NULL
    GROUP BY i.industry_name
)
    SELECT pan.industry_name
    FROM pan
    JOIN ae ON ae.industry_name = pan.industry_name
    WHERE pan_avg > ae_avg


Production and nonsupervisory employees did better on average in motor vehicle power train components AND upholstered housefold furniture than all employees in 2017 

3. Now that you have explored the datasets, is there any data or information that you wish you had in this analysis?

I was discussing this in Slack with a co-learner, but having the average aggregate values would have been helpful, particularly with question 1. I ended up calculating averages of averages which I know it not the cleanest, most optimal summary or representation of data. We had an aggregate (which I interpreted as the sum, aggregating all the values together) for certain values for employees, like hourly and weekly pay, and we had an average, but if there was an aggregate of all the averages I think that would have painted a much more clear and accurate picture. 